In [1]:
from header import *
#import matplotlib.pyplot as plt
#import mne
#import numpy as np
from scipy.stats.distributions import f,t
import visbrain
from visbrain.objects import BrainObj, ColorbarObj, SceneObj, SourceObj
from visbrain.io import read_stc
import xarray as xr

In [2]:
t0 = time.perf_counter()
task = 'SMEG' #'MIMOSA'
states = ['RS','FA','OM']
subjects = get_subjlist(task)

reject = ['004', '010']
for sub in reject:
    if sub in subjects:
        subjects.remove(sub)

subjects.sort()

experts = []
novices = []
for sub in subjects:
    if expertise(sub) == 'N':
        novices.append(sub)
    if expertise(sub) == 'E':
        experts.append(sub)

In [3]:
names = ['R_ECG_included', 'R_ECG_excluded', 'T_ECG_included', 'T_ECG_excluded']
noise_cov = 'baseline_cov'
fsaverage = '-fsaverage'#or False
stc_ext = '-lh.stc'
stc_path = op.join(Analysis_path, task, 'meg', 'SourceEstimate')

In [4]:
for n,name in enumerate(names[:1]):
    for t,state in enumerate(states[:1]):
        for s,sub in enumerate(subjects[:6]):
            stc_file = op.join(stc_path, sub, state+'*'+name+'*'+noise_cov+'*'+(fsaverage if fsaverage else '')+stc_ext)
            data = mne.read_source_estimate(glob.glob(stc_file)[0].strip(stc_ext))
            #data = np.random.rand(20484,1561)
            if not n and not t and not s:
                times = data.times
                vertices = np.concatenate([['lh_' + str(x) for x in data.lh_vertno],['rh_' + str(x) for x in data.rh_vertno]])
                stc = np.zeros((len(names), len(states), len(subjects), data.data.shape[1], data.data.shape[0]))
            stc[n,t,s] = data.data.T

AttributeError: 'numpy.ndarray' object has no attribute 'times'

In [ ]:
stc = xr.DataArray(stc, dims=['epoch', 'state', 'subject', 'src', 'time'], coords={'epoch':names, 'state':states, 'subject':subjects, 'time':times, 'src':vertices})

In [ ]:
def cluster_perm_test(X1, X2, p_threshold=0.01, spacing='ico5', paired=False):
    """
    If paired, test X2-X1.
    Input: arrays of shape (subjects, time, space)
    """
    X1 = np.abs(X1)
    X2 = np.abs(X2)
    
    f_threshold = f.ppf(1 - p_threshold / 2, X1.shape[0] - 1, X2.shape[0] - 1)
    t_threshold = t.ppf(p_threshold / 2, (X2-X1).shape[0] - 1)
    connectivity = mne.spatial_tris_connectivity(mne.grade_to_tris(int(spacing[-1])))
    
    if paired:
        return spatio_temporal_cluster_1samp_test((X2-X1), connectivity=connectivity, threshold=t_threshold, n_jobs=4)
    return spatio_temporal_cluster_test([X1,X2], connectivity=connectivity, threshold=f_threshold, n_jobs=4)

In [ ]:
#T_obs, clusters, cluster_pv, H0 = cluster_perm_test(stc.loc['R_ECG_included','RS'], stc.loc['R_ECG_included','FA'], paired=T)
#T_obs, clusters, cluster_pv, H0 = cluster_perm_test(stc.loc['R_ECG_included','RS',experts], stc.loc['R_ECG_included','FA',experts], paired=T)
#T_obs, clusters, cluster_pv, H0 = cluster_perm_test(stc.loc['R_ECG_included','RS',novices], stc.loc['R_ECG_included','FA',novices], paired=T)

#T_obs, clusters, cluster_pv, H0 = cluster_perm_test(stc.loc['R_ECG_included','RS',novices], stc.loc['R_ECG_included','RS',experts])
#T_obs, clusters, cluster_pv, H0 = cluster_perm_test(stc.loc['R_ECG_included','FA',novices]-stc.loc['R_ECG_included','RS',novices], stc.loc['R_ECG_included','FA',experts]-stc.loc['R_ECG_included','RS',experts])
